# Sora2 Watermark Remover (Google Colab版)

このノートブックは、Sora動画からウォーターマークを除去するツールのGoogle Colab版です。

**新機能:**
- `--frame-step`: フレーム処理間隔（1=全フレーム、2=隔フレーム...）
- `--target-fps`: 出力fps指定（0=入力と同じ）

**リポジトリ:** https://github.com/fulfulggg/Sora2WatermarkRemover

## 1. セットアップ

### GitHubからリポジトリをクローン

In [ ]:
# GitHubリポジトリのクローン
!git clone https://github.com/fulfulggg/Sora2WatermarkRemover.git
%cd Sora2WatermarkRemover

# 最新の機能ブランチに切り替え
!git checkout feat/frame-step

print("\n✅ リポジトリのクローンが完了しました。")

### ⚠️ 重要: Pillow依存関係の修正

**以下のセルを実行し、その後必ずランタイムを再起動してください。**

1. 下のセルを実行
2. 「ランタイム」→「ランタイムを再起動」を選択
3. 再起動後、次のセルから実行を続ける

In [ ]:
# Pillow依存関係の修正（実行後にランタイムを再起動してください）
!pip uninstall -y pillow PIL
!pip install -U "pillow==10.4.0"

# オプション: websockets警告の抑制
!pip install -U "websockets==15.0.1"

print("\n✅ 完了しました。ランタイム→ランタイムを再起動 を実行してください。")

In [ ]:
# 再起動後の確認（このセルを最初に実行）
import PIL
import PIL._util as u

print(f"Pillow version: {PIL.__version__}")  # => 10.4.0
print(f"is_directory exists: {hasattr(u, 'is_directory')}")  # => True

if PIL.__version__ == "10.4.0" and hasattr(u, "is_directory"):
    print("✅ Pillow依存関係は正常です。次のセルに進んでください。")
else:
    print("⚠️ 問題があります。ランタイムを再起動しましたか？")

In [ ]:
# 必要なパッケージのインストール
!pip install -q transformers torch opencv-python tqdm loguru iopaint
!apt-get -qq install -y ffmpeg

print("✅ パッケージのインストールが完了しました。")

## 2. デバイス確認

In [ ]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️ GPUが利用できません。ランタイム→ランタイムのタイプを変更→GPUを選択してください。")

## 3. 動画アップロード

In [ ]:
from google.colab import files

print("動画ファイルをアップロードしてください:")
uploaded = files.upload()

# アップロードされたファイルを取得
input_video = list(uploaded.keys())[0]
print(f"\nUploaded: {input_video}")

## 4. 動画処理

### パラメータ設定

- `max_bbox_percent`: バウンディングボックスの最大サイズ（画像に対する割合%）
- `frame_step`: フレーム処理間隔（1=全フレーム、2=隔フレーム...）
- `target_fps`: 出力fps（0=入力と同じ）

In [ ]:
# パラメータ設定（必要に応じて調整）
max_bbox_percent = 10.0  # バウンディングボックスの最大サイズ
frame_step = 1           # 1=全フレーム、2=隔フレーム
target_fps = 0.0         # 0=入力と同じ

output_video = "output_no_watermark.mp4"

# remwm.pyを実行
!python remwm.py "{input_video}" "{output_video}" \
  --max-bbox-percent {max_bbox_percent} \
  --frame-step {frame_step} \
  --target-fps {target_fps}

print(f"\n✅ 処理完了: {output_video}")

## 5. 結果のダウンロード

In [ ]:
# 結果のダウンロード
print("結果をダウンロード:")
files.download(output_video)

## 6. 使い方のヒント

### 処理速度を上げたい場合

```python
frame_step = 2  # 隔フレーム処理（約2倍速）
```

### 出力fpsを固定したい場合

```python
target_fps = 30  # 30fpsで出力
```

### 組み合わせ例

```python
frame_step = 2
target_fps = 30
# → 隔フレーム処理して30fpsで出力
```